## Similarity Exercise

In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from sentence_transformers import SentenceTransformer
import faiss

c:\Users\blond\anaconda3\envs\nlp_similarity\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this exercise, you've been provided the title and abstract of 500 recent machine learning research papers posted on arXiv.org.

In [3]:
articles = pd.read_csv('../data/arxiv_papers.csv')
articles.head()

,title,abstract,url
0,GoT-R1: Unleashing Reasoning Capability of MLL...,Visual generation models have made remarkable ...,http://arxiv.org/abs/2505.17022v1
1,Delving into RL for Image Generation with CoT:...,Recent advancements underscore the significant...,http://arxiv.org/abs/2505.17017v1
2,Interactive Post-Training for Vision-Language-...,"We introduce RIPT-VLA, a simple and scalable r...",http://arxiv.org/abs/2505.17016v1
3,When Are Concepts Erased From Diffusion Models?,"Concept erasure, the ability to selectively pr...",http://arxiv.org/abs/2505.17013v1
4,Understanding Prompt Tuning and In-Context Lea...,Prompting is one of the main ways to adapt a p...,http://arxiv.org/abs/2505.17010v1


In [4]:
i = 0
print(f'Title: {articles.loc[i,"title"]}\n')
print(f'Text: {articles.loc[i,"abstract"]}')

Title: GoT-R1: Unleashing Reasoning Capability of MLLM for Visual Generation with Reinforcement Learning

Text: Visual generation models have made remarkable progress in creating realistic
images from text prompts, yet struggle with complex prompts that specify
multiple objects with precise spatial relationships and attributes. Effective
handling of such prompts requires explicit reasoning about the semantic content
and spatial layout. We present GoT-R1, a framework that applies reinforcement
learning to enhance semantic-spatial reasoning in visual generation. Building
upon the Generation Chain-of-Thought approach, GoT-R1 enables models to
autonomously discover effective reasoning strategies beyond predefined
templates through carefully designed reinforcement learning. To achieve this,
we propose a dual-stage multi-dimensional reward framework that leverages MLLMs
to evaluate both the reasoning process and final output, enabling effective
supervision across the entire generation pipeli

Let's try out a variety of ways of vectorizing and searching for semantically-similar papers.

### Method 1: Bag of Words

Fit a CountVectorizer to the abstracts of the articles with all of the defaults.  Then vectorize the dataset using the fit vectorizer. 

In [19]:
cv = CountVectorizer()
cv.fit(articles['abstract'])
cv_tf = cv.transform(articles['abstract'])

**Question:** How many dimensions do the embeddings have?

In [20]:
cv_tf.shape

(500, 7978)

Now, let's use the embeddings to look for similar articles to a search query.

Apply the vectorizer you fit earlier to this query string to get an embedding. 

**Hint:** You can't pass a string to a vectorizer, but you can pass a list containing a string.

In [21]:
query = "vector databases for retrieval augmented generation"

cv_query = cv.transform([query])

In [24]:
cv_query.shape

(1, 7978)

Now, we need to find the similarity between our query embedding and each vectorized article.

For this, you can use the [cosine similarity function from scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)

Calculate the similarity between the query embedding and each article embedding and save the result to a variable named `similarity_scores`.

In [36]:
similarity_scores = cosine_similarity(cv_query, cv_tf)[0]

In [37]:
similarity_scores.shape

(500,)

Now, we need to find the most similar results. To help with this, we can use the [argsort function from numpy](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html), which will give the indices sorted by value. 

Use the argsort function to find the indices of the 5 most similar articles. Inspect their titles and abstracts. **Warning:** argsort sorts from smallest to largest.

In [39]:
indices = np.argsort(similarity_scores)

In [43]:
top_indicies = indices[::-1][:5]

In [44]:
top_indicies

array([259, 289,  83, 486, 394])

In [47]:
for i in top_indicies: 
    print(articles.loc[i, 'title'])
    print(articles.loc[i, 'abstract']) 
    print('---')

MIRB: Mathematical Information Retrieval Benchmark
Mathematical Information Retrieval (MIR) is the task of retrieving
information from mathematical documents and plays a key role in various
applications, including theorem search in mathematical libraries, answer
retrieval on math forums, and premise selection in automated theorem proving.
However, a unified benchmark for evaluating these diverse retrieval tasks has
been lacking. In this paper, we introduce MIRB (Mathematical Information
Retrieval Benchmark) to assess the MIR capabilities of retrieval models. MIRB
includes four tasks: semantic statement retrieval, question-answer retrieval,
premise retrieval, and formula retrieval, spanning a total of 12 datasets. We
evaluate 13 retrieval models on this benchmark and analyze the challenges
inherent to MIR. We hope that MIRB provides a comprehensive framework for
evaluating MIR systems and helps advance the development of more effective
retrieval models tailored to the mathematical domai

Try using a tfidf vectorizer. How do the results compare?

In [49]:
tfidf = TfidfVectorizer()
tfidf.fit(articles['abstract'])
tfidf_tf = tfidf.transform(articles['abstract'])

In [50]:
tfidf_tf.shape

(500, 7978)

In [53]:
tfidf_query = tfidf.transform([query])

In [54]:
similarity_scores_t = cosine_similarity(tfidf_query, tfidf_tf)[0]

In [56]:
similarity_scores_t.shape

(500,)

In [58]:
indices_t = np.argsort(similarity_scores_t)

In [59]:
top_indicies_t = indices_t[::-1][:5]

In [60]:
top_indicies_t 

array([259,  83, 289, 233, 486])

In [61]:
for i in top_indicies_t: 
    print(articles.loc[i, 'title'])
    print(articles.loc[i, 'abstract']) 
    print('---')

MIRB: Mathematical Information Retrieval Benchmark
Mathematical Information Retrieval (MIR) is the task of retrieving
information from mathematical documents and plays a key role in various
applications, including theorem search in mathematical libraries, answer
retrieval on math forums, and premise selection in automated theorem proving.
However, a unified benchmark for evaluating these diverse retrieval tasks has
been lacking. In this paper, we introduce MIRB (Mathematical Information
Retrieval Benchmark) to assess the MIR capabilities of retrieval models. MIRB
includes four tasks: semantic statement retrieval, question-answer retrieval,
premise retrieval, and formula retrieval, spanning a total of 12 datasets. We
evaluate 13 retrieval models on this benchmark and analyze the challenges
inherent to MIR. We hope that MIRB provides a comprehensive framework for
evaluating MIR systems and helps advance the development of more effective
retrieval models tailored to the mathematical domai

### Method 2: Using a Pretrained Embedding Model

Now, let's compare how we do using the [all-MiniLM-L6-v2 embedding model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

This will create a 384-dimensional dense embedding of each sentence.

In [62]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\blond\anaconda3\envs\nlp_similarity\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\blond\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fall

In [63]:
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings = embedder.encode(sentences)
print(embeddings)

[[ 6.76569641e-02  6.34958968e-02  4.87131327e-02  7.93049932e-02
   3.74479853e-02  2.65277526e-03  3.93749401e-02 -7.09840422e-03
   5.93614280e-02  3.15369591e-02  6.00980893e-02 -5.29051833e-02
   4.06068265e-02 -2.59308796e-02  2.98428163e-02  1.12684851e-03
   7.35149309e-02 -5.03819138e-02 -1.22386634e-01  2.37028524e-02
   2.97265034e-02  4.24768254e-02  2.56337989e-02  1.99519284e-03
  -5.69191016e-02 -2.71598455e-02 -3.29035446e-02  6.60248697e-02
   1.19007058e-01 -4.58790585e-02 -7.26214871e-02 -3.25839929e-02
   5.23413904e-02  4.50552516e-02  8.25305562e-03  3.67023498e-02
  -1.39415562e-02  6.53919354e-02 -2.64272839e-02  2.06388984e-04
  -1.36643872e-02 -3.62809822e-02 -1.95043311e-02 -2.89738625e-02
   3.94270457e-02 -8.84090886e-02  2.62431079e-03  1.36714112e-02
   4.83063310e-02 -3.11565101e-02 -1.17329173e-01 -5.11690043e-02
  -8.85288417e-02 -2.18962971e-02  1.42986458e-02  4.44168039e-02
  -1.34814847e-02  7.43392557e-02  2.66382769e-02 -1.98762212e-02
   1.79190

Use this new embedder to vectorize the abstracts and then find the most similar to the query. How do the results compare to the other methods?

**Warning:** Creating embeddings for all of the articles may take a while.

In [64]:
sent_emb = embedder.encode(articles['abstract'].tolist())

In [65]:
sent_emb.shape

(500, 384)

In [66]:
sent_query = embedder.encode([query])

In [67]:
similarity_scores_s = cosine_similarity(sent_query, sent_emb)[0]

In [69]:
indices_s = np.argsort(similarity_scores_s)

In [70]:
top_indicies_s = indices[::-1][:5]

In [71]:
top_indicies_s

array([259,  83, 289, 233, 486])

In [72]:
for i in top_indicies: 
    print(articles.loc[i, 'title'])
    # print(articles.loc[i, 'abstract']) 
    print('---')

MIRB: Mathematical Information Retrieval Benchmark
---
SCENIR: Visual Semantic Clarity through Unsupervised Scene Graph Retrieval
---
WikiDBGraph: Large-Scale Database Graph of Wikidata for Collaborative Learning
---
Explaining Neural Networks with Reasons
---
Scan, Materialize, Simulate: A Generalizable Framework for Physically Grounded Robot Planning
---


### FAISS

The [Faiss library](https://faiss.ai/index.html) is a library for efficient similarity search and clustering of dense vectors. It can be used to automate the process of finding the most similar abstracts.

If we want to use cosine similarity, we need to use the Inner Product. We also need to normalize our vectors so that they all have length 1.

Use the [normalize function](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html) to normalize both the abstract vectors and the query vector.

In [ ]:
# Your Code Here

Now, create an [IndexFlatIP object](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes#summary-of-methods) that has dimensions equal to the dimensionality of your vectors. Then add your normalized abstract vectors.

Hint: You can mimic the example [here](https://github.com/facebookresearch/faiss/wiki/Getting-started#building-an-index-and-adding-the-vectors-to-it), but substitute in the IndexFlatIP class.

In [ ]:
# Your Code Here

Finally, use the [search function](https://github.com/facebookresearch/faiss/wiki/Getting-started#searching) on your index object to find the 5 most similar articles.

In [ ]:
# Your Code Here